### Name(s):
- Ojas Patel (ovp74)
- Pranav Neravetla (prn289)
- Suhas Dara (sd35633)
- Avinash Damania (ad44428)

# OKCupid Data Mining Project

# Introduction

In this project, we will use an OKCupid dataset to solve the problem of predicting education level using information from dating profiles such as physical traits and lifestyle choices. Predicting someone's education level from their dating profile is useful for those with dating preferences. When making a profile, people will often avoid filling out certain fields, meaning that someone could match most of your preferences but be at a different stage of their education or career. For those who would prefer dating someone they can more strongly relate to in terms of school/work, predicting their education level can be quite valuable.

The OKCupid dataset we will use contains 19782 profiles from various residents of California. These profiles each have several different attributes that describe the person, such as age, body type, diet, and more. These attributes will be the basis of our predictive models.

In [1]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

# Some headers
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import model_selection, preprocessing, decomposition, neighbors, pipeline, tree, svm, multiclass
from sklearn import naive_bayes, neural_network, ensemble, metrics, linear_model

## Data Cleaning and Exploration

In this section, we will clean and engineer the data in preparation for use in training our models. Some of the tasks necessary include dropping unusable attributes, checking for missing values, feature engineering, removing outliers, and looking for potential class imbalances.

### Data Prep

For the sake of efficiency while developing our models, we used a downsampled data set from test_profiles.csv. Our

In the event that a row does not have a value for education, we must drop it, since we need labeled data for training testing. Otherwise, we form our data and label sets.

In [2]:
df = pd.read_csv("final_profiles.csv")
df.columns

# we must drop rows that do not have an education value to deal with missing values
df = df[df.education.notnull()]
df = df.reset_index(drop=True)

labels = df['education']
data = df.drop(columns=['education'])

data.head()

,Unnamed: 0,age,body_type,diet,drinks,drugs,essay0,essay1,essay2,essay3,...,location,offspring,orientation,pets,religion,sex,sign,smokes,speaks,status
0,0,47,fit,mostly anything,often,NaN,"i'm tall and slender, have a good creative job...",i'm trying to balance my innate sense of respo...,sending funny text messages. speling (of cours...,"i'm usually smiling. and as noted, i'm fairly ...",...,"san francisco, california","doesn&rsquo;t have kids, and doesn&rsquo;t wan...",straight,likes dogs and has cats,atheism and laughing about it,m,NaN,no,"english (fluently), spanish (poorly)",single
1,1,27,NaN,NaN,often,never,NaN,NaN,my job? thinking about dinner when i should re...,awkward as hell.,...,"san francisco, california",NaN,straight,NaN,atheism and laughing about it,f,NaN,no,english,seeing someone
2,2,43,skinny,NaN,socially,never,"very easy going:) love to read, photography is...",NaN,"conversation, photography, writing, cooking, s...",my glasses:),...,"alameda, california",doesn&rsquo;t want kids,gay,has dogs,judaism but not too serious about it,f,sagittarius,no,english,single
3,3,22,NaN,mostly anything,NaN,NaN,i am always wanting to try new things--new act...,i about to graduate from cal as psych major an...,picking up new activities to try out and learn...,my smile! my friends say i seem happy about ev...,...,"berkeley, california",doesn&rsquo;t have kids,straight,likes dogs,NaN,m,leo but it doesn&rsquo;t matter,NaN,"english (fluently), chinese (okay)",single
4,4,31,fit,mostly anything,socially,never,"i work out somewhat regularly, dark hair and e...",i am gainfully employed and work as a project ...,i am good at making people laugh and feel comf...,"blunt, unapologetic honesty. i don't understan...",...,"oakland, california",doesn&rsquo;t have kids,straight,has dogs and likes cats,christianity but not too serious about it,m,cancer and it&rsquo;s fun to think about,no,english (fluently),single


In [3]:
len(data)

17560

In [4]:
data.count()

Unnamed: 0     17560
age            17560
body_type      16104
diet           10797
drinks         16942
drugs          13494
essay0         16115
essay1         15616
essay2         15001
essay3         14415
essay4         14839
essay5         14718
essay6         13821
essay7         14198
essay8         12237
essay9         14114
ethnicity      16056
height         17560
income         17560
job            15898
last_online    17560
location       17560
offspring       7409
orientation    17560
pets           12199
religion       12182
sex            17560
sign           14733
smokes         16236
speaks         17547
status         17560
dtype: int64

There are several columns that we found necessary to drop after examining the data manually. The reasons are listed below:

- Removed 'Unnamed' because it is just a repeat of the index value
- Removed essay0, essay1, essay2, essay3, essay4, essay5, essay6, essay7, essay8, and essay9 because they are paragraph answers and so every row will have a unique value (if we wanted to use this, we would have to do sentiment analysis or something similar)
- Removed 'last_online' as it can't be used to predict the label (education)
- Removed 'sign' as it can't be used to predict the label (education)
- Removed 'offspring' as too many rows have NaN as a value
- Removed 'diet' as too many rows have NaN as a value
- Removed 'speaks' as there are too many distinct values and cannot be mapped into a smaller domain
- Removed 'location' as there art too many distinct values and it cannot be used to predict the label (educatin)
- Removed 'status' as there is a heavy class imbalance with almost all of the values being 'single'
- Removed 'income' as almost all values are not listed (value appears as -1)

In [5]:
data = data.drop(columns = ['Unnamed: 0', 'essay0', 'essay1','essay2','essay3','essay4','essay5','essay6','essay7'
                  ,'essay8','essay9','last_online','sign','offspring', 'diet', 'speaks','location','status', 'income'],axis=0)

After exploring and removing those columns from the data, we can now take a look at the columns that we kept, as well as the labels. We found that a lot of values would overlap, or there would just be too many possible values to glean meaningful information from someone's answer. Therefore, most of the columns needed to map values to a smaller subset of values. We start with our labels here, refining them to the values we think would be valuable to predict. As a side note, 'space camp' here is a joke value that OKCupid allows users to select.

In [6]:
# dictionary to consolidate the labels for education
label_engineering = {
    'graduated from college/university': 'bachelors',
    'greatued from masters program': 'advanced degree',
    'working on college/university': 'bachelors',
    'working on masters program': 'advanced degree',
    'graduated from two-year college': 'associates',
    'graduated from high school': 'high-school',
    'graduated from ph.d program': 'advanced degree',
    'graduated from law school': 'advanced degree',
    'working on two-year college': 'associates',
    'working on ph.d program': 'advanced degree',
    'dropped out of college/university': 'high-school',
    'college/university': 'bachelors',
    'graduated from space camp': 'spacecamp',
    'dropped out of space camp': 'spacecamp',
    'graduated from med school': 'advanced degree',
    'working on space camp': 'spacecamp',
    'working on law school': 'advanced degree',
    'working on med school': 'advanced degree',
    'dropped out of two-year college': 'high-school',
    'two-year college': 'associates',
    'masters program': 'advanced degree',
    'dropped out of masters program': 'advanced degree',
    'dropped out of ph.d program': 'advanced degree',
    'high school': 'high-school',
    'dropped out of high school': 'high-school',
    'working on high school': 'high-school',
    'space camp': 'spacecamp',
    'ph.d program': 'advanced degree',
    'med school': 'advanced degree',
    'law school': 'advanced degree',
    'dropped out of law school': 'advanced degree',
    'dropped out of med school': 'advanced degree',
    'graduated from masters program': 'advanced degree'}

In [7]:
labels = labels.replace(label_engineering)

In [8]:
labels.value_counts()

bachelors          9990
advanced degree    5122
associates          953
high-school         946
spacecamp           549
Name: education, dtype: int64

Now that we have consolidated our education labels, we can move on to our dataset, where we will need to fill in missing values and consolidate the range of values for each feature.

In [9]:
data.count()

age            17560
body_type      16104
drinks         16942
drugs          13494
ethnicity      16056
height         17560
job            15898
orientation    17560
pets           12199
religion       12182
sex            17560
smokes         16236
dtype: int64

The columns for age, sex, and orientation are all full, so we won't need to fill in missing values, but we will still check the values and do the mappings. We start with the age column, and upon inspection it appears to be fine.

In [10]:
data['age'].value_counts()

27    1113
26    1100
28    1076
25    1035
29    1011
24     979
30     906
23     805
32     791
31     786
33     651
22     561
34     559
35     494
36     471
37     403
38     383
21     348
39     328
40     308
42     283
20     270
41     269
43     230
44     220
45     182
46     169
19     159
48     157
47     147
49     132
50     120
51     106
52      96
56      89
55      82
57      82
54      74
18      69
53      66
58      62
59      58
60      58
63      51
61      48
62      43
64      33
65      31
66      27
67      16
68      13
69      10
Name: age, dtype: int64

We now look at the unique values in the 'body_type' column and fill in missing values.

In [11]:
data['body_type'].value_counts()

average           4179
fit               3869
athletic          3526
thin              1397
curvy             1143
a little extra     767
skinny             504
full figured       307
overweight         133
jacked             113
used up            111
rather not say      55
Name: body_type, dtype: int64

About 6% of the elements are missing a value for body_type. Given that 'average' is the mode of all the values and that it is safe to assume that the typical person has an "average" body type, we will fill the NaNs with 'average'.

In [12]:
# dictionary to consolidate the values for the body_type column because a lot of them are redundant
body_type_dictionary = {
    'average': 'average',
    'athletic': 'athletic',
    'fit': 'athletic',
    'thin': 'underweight',
    'curvy': 'overweight',
    'a little extra': 'overweight',
    'skinny': 'underweight',
    'full figured': 'overweight',
    'jacked': 'athletic',
    'overweight': 'overweight',
    'used up': 'overweight',
    'rather not say': 'average'
}

data['body_type'] = data['body_type'].fillna('average')
data['body_type'] = data['body_type'].replace(body_type_dictionary)
data['body_type'].value_counts()

athletic       7508
average        5690
overweight     2461
underweight    1901
Name: body_type, dtype: int64

We now check unique values for the 'drinks' column and fill in missing values.

In [13]:
data['drinks'].value_counts()

socially       12512
rarely          1777
often           1508
not at all       948
very often       112
desperately       85
Name: drinks, dtype: int64

Since the values are categorical, we can't take an average. Instead, we check the mode, which in this column is "socially". The mode makes up about 70% of the total values in this column, so we will fill all NaNs with "socially."

In [14]:
# dictionary to consolidate the values for the drinks column because a lot of them are redundant
drinks_dictionary = {
    'socially': 'sometimes',
    'often': 'often',
    'rarely': 'rarely',
    'not at all': 'never',
    'desperately': 'very often',
    'very often': 'very often'
}

data['drinks'] = data['drinks'].fillna('socially')
data['drinks'] = data['drinks'].replace(drinks_dictionary)
data['drinks'].value_counts()

sometimes     13130
rarely         1777
often          1508
never           948
very often      197
Name: drinks, dtype: int64

We now check unique values for the 'drugs' column and fill in missing values

In [15]:
data['drugs'].value_counts()

never        11066
sometimes     2311
often          117
Name: drugs, dtype: int64

Since the values are categorical once again, we will use the mode, which is "never" for this column. It makes up for about 80% of reported values.

In [16]:
data['drugs'] = data['drugs'].fillna('never')
data['drugs'].value_counts()

never        15132
sometimes     2311
often          117
Name: drugs, dtype: int64

We now check unique values for the 'ethnicity' column and fill in missing values. It is important to note here that there seem to be some records in our data that seem to be pranks. There are records that select all the available ethnicity values, this has a very very small probability is reality.

In [17]:
# we will check unique values for ethnicity and fill in missing values
data['ethnicity'].value_counts()

white                                                                                               9769
asian                                                                                               1835
hispanic / latin                                                                                     839
black                                                                                                578
other                                                                                                486
                                                                                                    ... 
asian, black, native american, hispanic / latin                                                        1
asian, black, hispanic / latin, other                                                                  1
asian, middle eastern, black, native american, indian, pacific islander, hispanic / latin, other       1
asian, black, native american, hispanic / latin, white 

We must again use the mode to fill in missing values for this categorical feature, which ends up being "white", making up about 50% of the total data. There was a lot of diversity in ethnicity, so we narrowed down the categories to white, asian, black, hispanic, native, pacific, mixed, and other. If only the second ethnicity is 'other', that means it might be insignificant enough to be ignored, and the row gets classified as the first ethnicity reported.

In [18]:
ethnicity_dictionary = {
    'white': 'white',
    'asian': 'asian',
    'hispanic / latin': 'hispanic',
    'black': 'black',
    'other': 'other',
    'indian': 'asian',
    'white, other': 'white',
    'middle eastern': 'asian',
    'black, other': 'black',
    'hispanic / latin, other': 'hispanic',
    'pacific islander': 'pacific',
    'asian, other': 'asian',
    'native american': 'native',
    'indian, other': 'asian',
    'native american, other': 'native',
    'pacific islander, other': 'pacific',
    'middle eastern, other': 'asian'
}

data['ethnicity'] = data['ethnicity'].fillna('white')
data['ethnicity'] = data['ethnicity'].replace(ethnicity_dictionary)

data.loc[~data['ethnicity'].isin(['white', 'asian', 'hispanic', 'black', 'pacific', 'native', 'other']), 'ethnicity'] = 'mixed'
data['ethnicity'].value_counts()

white       11480
asian        2281
mixed        1684
hispanic      880
black         622
other         486
pacific       107
native         20
Name: ethnicity, dtype: int64

For the 'height' column, we decide to fill in missing values using the average height of the row's respective gender.

In [19]:
data['height'] = df['height'].fillna(df.groupby('sex')['height'].transform('mean'))

We now check unique values for the 'jobs' column and fill in missing values.

In [20]:
data['job'].value_counts()

other                                2246
student                              1566
science / tech / engineering         1515
computer / hardware / software       1467
artistic / musical / writer          1343
sales / marketing / biz dev          1340
medicine / health                    1147
education / academia                 1134
executive / management                730
entertainment / media                 720
banking / financial / real estate     687
law / legal services                  432
hospitality / travel                  396
construction / craftsmanship          260
clerical / administrative             240
political / government                213
rather not say                        128
transportation                        108
unemployed                             87
retired                                76
military                               63
Name: job, dtype: int64

Since the values are categorical, and quite a few people already did not want to reveal their job anyways, the empty fields are also clumped into the 'rather not say' category. We narrowed down the categories to student, STEM, arts, business, education, not working, and military because there were a lot of different jobs and a lot of them fall into an overlying category.

In [21]:
job_dictionary = {
    'other': 'other',
    'student': 'student',
    'science / tech / engineering': 'STEM',
    'computer / hardware / software': 'STEM',
    'artistic / musical / writer': 'arts',
    'sales / marketing / biz dev': 'business',
    'education / academia': 'education',
    'medicine / health': 'STEM',
    'banking / financial / real estate': 'business',
    'executive / management': 'business',
    'hospitality / travel': 'business',
    'entertainment / media': 'arts',
    'law / legal services': 'arts',
    'clerical / administrative': 'business',
    'political / government': 'arts', 
    'construction / craftsmanship': 'STEM',
    'rather not say': 'other',
    'transportation': 'STEM',
    'unemployed': 'not working',
    'retired': 'not working',
    'military': 'military'
}

data['job'] = data['job'].fillna('rather not say')
data['job'] = data['job'].replace(job_dictionary)
data['job'].value_counts()

STEM           4497
other          4036
business       3393
arts           2708
student        1566
education      1134
not working     163
military         63
Name: job, dtype: int64

We now check unique values for the 'pets' column and fill in missing values.

In [22]:
data['pets'].value_counts()

likes dogs and likes cats          4582
likes dogs                         2175
likes dogs and has cats            1333
has dogs                           1286
has dogs and likes cats             685
likes dogs and dislikes cats        620
has dogs and has cats               426
has cats                            403
likes cats                          307
has dogs and dislikes cats          149
dislikes dogs and likes cats         94
dislikes dogs and dislikes cats      55
dislikes cats                        37
dislikes dogs and has cats           29
dislikes dogs                        18
Name: pets, dtype: int64

There are a lot of different combinations of liking, disliking or owning dogs or cats, so we narrowed it down to simply pets rather than dogs or cats. We also fill in NaNs with 'likes' as the average person does not mind pets (but we do not want to assume ownership).

In [23]:
pet_dictionary = {
    'likes dogs and likes cats': 'likes',
    'likes dogs': 'likes',
    'likes dogs and has cats': 'owns',
    'has dogs': 'owns',
    'has dogs and likes cats': 'owns',
    'likes dogs and dislikes cats': 'likes',
    'has cats': 'owns',
    'has dogs and has cats': 'owns',
    'likes cats': 'likes',
    'dislikes dogs and dislikes cats': 'dislikes',
    'has dogs and dislikes cats': 'owns',
    'dislikes dogs and likes cats': 'likes',
    'dislikes cats': 'dislikes',
    'dislikes dogs and has cats': 'owns',
    'dislikes dogs': 'dislikes'
}

data['pets'] = data['pets'].fillna('likes')
data['pets'] = data['pets'].replace(pet_dictionary)
data['pets'].value_counts()

likes       13139
owns         4311
dislikes      110
Name: pets, dtype: int64

We now check the 'religion' column for unique values and fill in missing values.

In [24]:
data['religion'].value_counts()

agnosticism                                   855
agnosticism but not too serious about it      812
other                                         808
agnosticism and laughing about it             793
catholicism but not too serious about it      707
other and laughing about it                   676
atheism                                       626
atheism and laughing about it                 623
christianity but not too serious about it     605
christianity                                  561
other but not too serious about it            467
judaism but not too serious about it          466
atheism but not too serious about it          405
catholicism                                   322
christianity and somewhat serious about it    303
atheism and somewhat serious about it         258
other and somewhat serious about it           255
catholicism and laughing about it             237
agnosticism and somewhat serious about it     201
judaism and laughing about it                 198


The religion data in this dataset also includes the mentality of the person in terms with there religiousness. Ignoring the religious mentality, we narrowed down to the different religions (or lackthereof). We also filled the NaNs with not-listed and clumped it with 'other'.

In [25]:
# dictionary to consolidate values for the religion column because there were a lot of redundant values
religion_dictionary = {    
    "agnosticism but not too serious about it": "agnostic",
    "agnosticism": "agnostic",
    "agnosticism and laughing about it": "agnostic",
    "other": "not-listed",
    "atheism": "atheist",
    "other and laughing about it": "not-listed",
    "christianity": "christian",
    "catholicism but not too serious about it": "christian",
    "atheism and laughing about it": "atheist",
    "christianity but not too serious about it": "christian",
    "atheism but not too serious about it": "atheist",
    "other but not too serious about it": "not-listed",
    "judaism but not too serious about it": "jewish",
    "catholicism": "christian",
    "other and somewhat serious about it": "not-listed",
    "catholicism and laughing about it": "christian",
    "christianity and somewhat serious about it": "christian",
    "atheism and somewhat serious about it": "atheist",
    "buddhism and laughing about it": "buddhist",
    "agnosticism and somewhat serious about it": "agnostic",
    "judaism and laughing about it": "jewish",
    "judaism": "jewish",
    "atheism and very serious about it": "atheist",
    "catholicism and somewhat serious about it": "christian",
    "buddhism but not too serious about it": "buddhist",
    "buddhism": "buddhist",
    "christianity and very serious about it": "christian",
    "other and very serious about it": "not-listed",
    "christianity and laughing about it": "christian",
    "agnosticism and very serious about it": "agnostic",
    "buddhism and somewhat serious about it": "buddhist",
    "hinduism but not too serious about it": "hindu",
    "judaism and somewhat serious about it": "jewish",
    "catholicism and very serious about it": "christian",
    "hinduism and somewhat serious about it": "hindu",
    "buddhism and very serious about it": "buddhist",
    "hinduism": "hindu",
    "islam": "muslim",
    "islam but not too serious about it": "muslim",
    "islam and very serious about it": "muslim",
    "hinduism and laughing about it": "hindu",
    "islam and somewhat serious about it": "muslim",
    "hinduism and very serious about it": "hindu",
    "judaism and very serious about it": "jewish",
    "islam and laughing about it": "muslim"
}

data['religion'] = data['religion'].fillna('not-listed')
data['religion'] = data['religion'].replace(religion_dictionary)
data['religion'].value_counts()

not-listed    7755
christian     3246
agnostic      2766
atheist       2093
jewish         935
buddhist       592
hindu          138
muslim          35
Name: religion, dtype: int64

We now check unique values for the 'smokes' column and fill in missing values.

In [26]:
# we will check unique values for smokes and fill in missing values
data['smokes'].value_counts()

no                13313
sometimes          1072
when drinking       874
yes                 568
trying to quit      409
Name: smokes, dtype: int64

The mode for this column is 'no', making up about 75% of the data. We accordingly fill in Nans with 'no'.

In [27]:
# 'no' is the mode and makes up about 75% of the data, so we will fill NaNs with 'no'
data['smokes'] = data['smokes'].fillna('no')
data['smokes'].value_counts()

no                14637
sometimes          1072
when drinking       874
yes                 568
trying to quit      409
Name: smokes, dtype: int64

### Feature Engineering

We can convert some of the categorical features into numerical values because they are on an ordinal scale. This allows there to be a larger difference between values that are further away. For example, in the drinks column, 'often' is further from 'never' than 'sometimes' is. This conversion to numerical values for ordinal features allows us to incorporate this magnitude in difference rather than having the standard distance of 1 between different categorical features. We use ordinal features for all of the models we tried.

In [28]:
# converts some of the categorical features into values because they are ordinal
def convert_ordinal(data):
    copy = data.copy()
    
    drugs_codes = {
        'never': 0,
        'sometimes': 1,
        'often': 2
    }

    drinks_codes = {
        'never': 0,
        'rarely': 1,
        'sometimes': 2,
        'often': 3,
        'very often': 4
    }

    smokes_codes = {
        "no": 0,
        "when drinking": 1,
        "trying to quit": 2,
        "sometimes": 3,
        "yes": 4
    }

    pets_codes = {
        "dislikes": 0,
        "likes": 1,
        "owns": 2
    }
    
    body_codes = {
        "underweight": 0,
        "average": 1,
        "athletic": 2,
        "overweight": 3
    }

    copy['drugs'] = data['drugs'].replace(drugs_codes)
    copy['drinks'] = data['drinks'].replace(drinks_codes)
    copy['smokes'] = data['smokes'].replace(smokes_codes)
    copy['pets'] = data['pets'].replace(pets_codes)
    copy['body_type'] = data['body_type'].replace(body_codes)
    return copy

data = convert_ordinal(data)
data.head()

,age,body_type,drinks,drugs,ethnicity,height,job,orientation,pets,religion,sex,smokes
0,47,2,3,0,white,75.0,other,straight,2,atheist,m,0
1,27,1,3,0,white,64.0,other,straight,1,atheist,f,0
2,43,0,2,0,white,63.0,other,gay,2,jewish,f,0
3,22,1,2,0,asian,71.0,student,straight,1,not-listed,m,0
4,31,2,2,0,mixed,66.0,STEM,straight,2,christian,m,0


Most of the models that we plan on using do not accept categorical data. Therefore, for the categorical features that are nominal and not ordinal, we are using one-hot encoding where applicable to be able to use these features.

In [29]:
def one_hot_encode(data, column=None):
    copy = data.copy()
    
    if column is None:
        to_encode = ['ethnicity', 'job', 'orientation', 'religion', 'sex']
        for column in to_encode:
            copy = one_hot_encode(copy, column)
    else:
        dummies = pd.get_dummies(copy[[column]])
        copy = pd.concat([copy, dummies], axis=1)
        copy = copy.drop([column], axis=1)
        
    return copy

one_hot_encode(data).head()

,age,body_type,drinks,drugs,height,pets,smokes,ethnicity_asian,ethnicity_black,ethnicity_hispanic,...,religion_agnostic,religion_atheist,religion_buddhist,religion_christian,religion_hindu,religion_jewish,religion_muslim,religion_not-listed,sex_f,sex_m
0,47,2,3,0,75.0,2,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
1,27,1,3,0,64.0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
2,43,0,2,0,63.0,2,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
3,22,1,2,0,71.0,1,0,1,0,0,...,0,0,0,0,0,0,0,1,0,1
4,31,2,2,0,66.0,2,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1


### More Data Exploration

Some of the models that we plan on using are very sensitive to the effect of outliers, so we have to remove them in order for the models to function effectively. We are utilizing isolation forests with a 5% contamination rate to remove outliers.

In [30]:
def remove_outliers(data, labels):
    data_cp = data.copy()
    labels_cp = labels.copy()
    
    iso_forest = ensemble.IsolationForest(n_estimators=100, contamination=0.05)
    pred = iso_forest.fit_predict(data_cp.drop(columns=['ethnicity', 'job', 'orientation', 'religion', 'sex']))
    pred = pd.Series(pred)
    data_cp = data_cp[pred == 1].reset_index(drop=True)
    labels_cp = labels_cp[pred == 1].reset_index(drop=True)
    
    return (data_cp, labels_cp)

data_cp, labels_cp = remove_outliers(data, labels)
print(data_cp.head(), '\n')
print(labels_cp.head())

   age  body_type  drinks  drugs ethnicity  height      job orientation  pets  \
0   47          2       3      0     white    75.0    other    straight     2   
1   27          1       3      0     white    64.0    other    straight     1   
2   43          0       2      0     white    63.0    other         gay     2   
3   22          1       2      0     asian    71.0  student    straight     1   
4   31          2       2      0     mixed    66.0     STEM    straight     2   

     religion sex  smokes  
0     atheist   m       0  
1     atheist   f       0  
2      jewish   f       0  
3  not-listed   m       0  
4   christian   m       0   

0          bachelors
1          bachelors
2    advanced degree
3          bachelors
4          bachelors
Name: education, dtype: object


We also wanted to try balanced data for certain models due to there being too few of certain labels. We upsample and downsample accordingly in an attempt to even out any class imbalances.

In [31]:
def balance_data(data, labels, rows=None):
    if rows is None:
        rows = len(data.index) // len(labels.unique())
    
    all_data = pd.concat([data, labels], axis=1)
    unique_labels = labels.unique()
    
    to_concat = []
    for label in unique_labels:
        num_label = len(all_data[all_data['education']==label].index)
        if num_label > rows:
            #downsample
            down = all_data[all_data['education']==label].sample(n=rows)
            to_concat.append(down)
        elif num_label < rows:
            #upsample
            choices = all_data[all_data['education']==label]
            indices = np.array(choices.index)
            resampled_indices = np.random.choice(indices, size=rows, replace=True)
            up = choices.loc[resampled_indices]
            to_concat.append(up)
    
    new_df = pd.concat(to_concat, axis=0).sample(frac=1).reset_index(drop=True)
    new_labels = new_df['education']
    new_data = new_df.drop(columns=['education'])
    
    return new_data, new_labels

data_balanced, labels_balanced = balance_data(data, labels)
print(data_balanced.head(), '\n')
print(labels_balanced.head())

   age  body_type  drinks  drugs ethnicity  height    job orientation  pets  \
0   32          2       2      0     white    70.0  other    straight     1   
1   35          2       2      0     white    68.0   arts    straight     1   
2   20          0       3      0     white    66.0   arts    straight     1   
3   41          2       2      0     white    71.0   arts    straight     2   
4   36          2       2      1     white    75.0   STEM    straight     1   

     religion sex  smokes  
0  not-listed   m       0  
1  not-listed   f       0  
2  not-listed   f       0  
3  not-listed   m       0  
4  not-listed   m       3   

0    advanced degree
1    advanced degree
2         associates
3          spacecamp
4          spacecamp
Name: education, dtype: object


## Modeling

### K-Nearest Neighbors

The first model we tried was a KNN classifier. It's a common classifier and we decided to start out seeing what accuracy we could get with it. KNN could be sensitive to outliers because these points may be very far from most data points distance wise, and may not be classified correctly. Additionally, KNN breaks down in higher dimensionality, so we opted not to one hot encode the nominal features and instead completely drop them. Lastly, we used a balanced dataset to account for the class imbalance. The class imbalance is not good for the KNN, as there will probabilistically be more neighbors of the dominant label.

In [32]:
scaler = preprocessing.StandardScaler()
pca = decomposition.PCA()
knn = neighbors.KNeighborsClassifier()
ppl = pipeline.Pipeline(steps=[('scaler', scaler), ('pca', pca), ('knn', knn)])

knn_data, knn_labels = remove_outliers(data, labels)
knn_data = knn_data.drop(columns=['ethnicity', 'job', 'orientation', 'religion', 'sex'])
knn_data, knn_labels = balance_data(knn_data, knn_labels)

param_grid = {'pca__n_components': [val / 20 for val in range(12, 20)], 'knn__n_neighbors': list(range(5, 10))}
inner = model_selection.GridSearchCV(ppl, param_grid, scoring='accuracy', cv=5, iid=True)
pred = model_selection.cross_val_predict(inner, knn_data, knn_labels, cv=5)
print('Accuracy:', metrics.accuracy_score(knn_labels, pred))
print('Weighted F1:', metrics.f1_score(knn_labels, pred, average='weighted'))

Accuracy: 0.5887290167865707
Weighted F1: 0.5670520939031272


The balancing of data does introduce one problem -- the final accuracy evaluated will be slightly skewed to the higher side because of the upsampled data contributing many more true positives and true negatives than they are supposed to. This is probably avoidable if the data is trained according to the balanced dataset, but evaluated against the original dataset. However, the KNN produced the highest accuracy (68%) and F1 score (0.64) of all the models we tried when we ran on the sample data set (1782 records), but on the overall data, it seems to be performing similar to the rest of our models, with the exception of a slightly better F1 score.

### Decision Tree

We decided to try a decision tree next, as they can handle categorical data better as they can be split along the unique values. Decision trees do not break down in high dimensionality, which is a benefit for us as we have a large number of features that get introduced because of the one-hot encoding. One-hot encoding performed better than altogether removing the nominal features, which indicates that there is some information in these features that could be extracted by classifiers that can better handle them.

In [33]:
tree_data = one_hot_encode(data)

param_grid = {'criterion': ['gini', 'entropy'], 'splitter': ['best', 'random'], 'min_samples_leaf': [val / 200 for val in range(1, 11)]}
inner = model_selection.GridSearchCV(tree.DecisionTreeClassifier(), param_grid, scoring='accuracy', cv=5, iid=True)
pred = model_selection.cross_val_predict(inner, tree_data, labels, cv=5)
print('Accuracy:', metrics.accuracy_score(labels, pred))
print('Weighted F1:', metrics.f1_score(labels, pred, average='weighted'))

Accuracy: 0.5847949886104784
Weighted F1: 0.5019257731825983


Using just the accuracy of the decision tree seemed to provide false information. We got an accuracy between 56% and 58% on average. However the weighted F1 score (weighted to account for the class imbalance) was much lower at around 48-50%. This indicates that either our precision, recall, or both, especially for the dominant class (i.e. bachelors), is not very good.

### SVMs

For SVMs, we tried multiple different models, including SVC and LinearSVC, in combination with both one vs one approach, and one vs rest approach, with or without balanced data. We decided to remove outliers for the SVMs because they will greatly influence the cost function, which is not something we would want. Additionally, we chose not to one hot encode the data because of the curse of dimensionality also applying to SVMs.

<B>Note</B>: SVMs were only run using the test set. The SVMs took almost 2 hours to run just with the test set. With the accuracy that we observed (as seen below), we felt it was not worth the time to run it on the final data set. 

In [40]:
def run_multiclass_svm_model(data, labels, outer_model, name):
    scaler = preprocessing.StandardScaler()
    pca = decomposition.PCA()
    ppl = pipeline.Pipeline(steps=[('scaler', scaler), ('pca', pca), ('clf', outer_model)])
    
    svm_data, svm_labels = remove_outliers(data, labels)
    svm_data = svm_data.drop(columns=['ethnicity', 'job', 'orientation', 'religion', 'sex'])

    inner_clfs = [svm.SVC(kernel=type, class_weight='balanced') for type in ['linear', 'poly', 'sigmoid', 'rbf']]
    inner_clfs.extend([svm.LinearSVC(loss=type, class_weight='balanced') for type in ['hinge', 'squared_hinge']])
    param_grid = {'pca__n_components': [val / 10 for val in range(6, 10)], 'clf__estimator': inner_clfs}
    
    inner = model_selection.GridSearchCV(ppl, param_grid, scoring='accuracy', cv=5, iid=True)
    pred = model_selection.cross_val_predict(inner, svm_data, svm_labels, cv=5)
    print(name, 'accuracy:', metrics.accuracy_score(svm_labels, pred))
    print(name, 'weighted F1:', metrics.f1_score(svm_labels, pred, average='weighted'))

run_multiclass_svm_model(data, labels, multiclass.OneVsOneClassifier(None), 'One vs One')
run_multiclass_svm_model(data, labels, multiclass.OneVsRestClassifier(None), 'One vs Rest')

One vs One accuracy: 0.35815602836879434
One vs One weighted F1: 0.36026139063452167
One vs Rest accuracy: 0.38416075650118203
One vs Rest weighted F1: 0.42131740872006374


In [41]:
def run_svc_model(data, labels, balanced):
    weight = 'balanced' if balanced else None
    score = 'accuracy' if balanced else 'f1_weighted'

    scaler = preprocessing.StandardScaler()
    pca = decomposition.PCA()
    svc = svm.SVC(class_weight=weight)
    ppl = pipeline.Pipeline(steps=[('scaler', scaler), ('pca', pca), ('svm', svc)])

    svm_data, svm_labels = remove_outliers(data, labels)
    svm_data = svm_data.drop(columns=['ethnicity', 'job', 'orientation', 'religion', 'sex'])

    kernels = ['linear', 'poly', 'sigmoid', 'rbf']
    costs = [1.0, 10.0, 100.0]
    param_grid = {'svm__kernel': kernels, 'svm__C': costs, 'svm__decision_function_shape': ['ovo', 'ovr']}

    inner = model_selection.GridSearchCV(ppl, param_grid, scoring=score, cv=5, iid=True)
    pred = model_selection.cross_val_predict(inner, svm_data, svm_labels, cv=5)
    
    name = 'Balanced' if balanced else 'Unbalanced'
    print(name, 'SVC accuracy', metrics.accuracy_score(svm_labels, pred))
    print(name, 'SVC weighted F1', metrics.f1_score(svm_labels, pred, average='weighted'))

run_svc_model(data, labels, True)
run_svc_model(data, labels, False)

Balanced SVC accuracy 0.36997635933806144
Balanced SVC weighted F1 0.40520373244710955
Unbalanced SVC accuracy 0.5236406619385343
Unbalanced SVC weighted F1 0.4829922891879866


We found that the SVC accuracy on balanced data was significantly lower than on unbalanced data. Balancing the data upsampled or downsampled data from certain classes based on the parameter. Given the heavy class imbalance favoring bachelors, it makes sense why a balanced SVM that used data that downsampled bachelors data and upsampled data from other classes would have a lower accuracy as now it could be overfitting to elements from underrepresented classes, causing those to be predicted more. 

The unbalanced SVC had a slightly lower accuracy than the neural network and the decision tree, but was still worse than the KNN model. However, the downside of the SVC was that it took very long to run. As mentioned above, it took about 2 hours to run on our smaller test dataset (1782 records). This is likely because of the multiclass aspect of our dataset. It is harder for the SVM algorithm to find concurrent hyperplanes to divide the data into multiple hyperspaces than to find a single hyperspace. SVM does not seem to be a good choice for multiclass problems where data is not well clustered.

### Naive Bayes

We also wanted to try a naive bayes model. Since we don't know the distribution of the data, we tried both GaussianNB and MultinomialNB. The model accuracy was understandably extremely poor with one hot encoding because of too many small probabilities introduced with the new one hot encoded features, so we just dropped the nominal features completely. There are no specific hyperparameters of interest, so we chose not to perform a GridSearchCV on the naive bayes algorithms.

In [34]:
def run_naive_bayes(data, labels, isGaussianNB):
    if isGaussianNB:
        classifier = naive_bayes.GaussianNB()
        classifier_roc_curve = naive_bayes.GaussianNB()
    else:
        classifier = naive_bayes.MultinomialNB()
        classifier_roc_curve = naive_bayes.MultinomialNB()
    
    bayes_data = data.drop(columns=['ethnicity', 'job', 'orientation', 'religion', 'sex'])
    
    pred = model_selection.cross_val_predict(classifier, bayes_data, labels, cv=10)
    
    name = 'GaussianNB' if isGaussianNB else 'MultinomialNB'
    print(name, 'accuracy:', metrics.accuracy_score(labels, pred))
    print(name, 'weighted F1:', metrics.f1_score(labels, pred, average='weighted'))

run_naive_bayes(data, labels, True)
run_naive_bayes(data, labels, False)

GaussianNB accuracy: 0.5314350797266515
GaussianNB weighted F1: 0.5028811553148655
MultinomialNB accuracy: 0.539123006833713
MultinomialNB weighted F1: 0.4959527808210238


GaussianNB is typically used when features have a normal distribution and are continuous. In the case of this dataset, the data isn't really continous except for features such as height, so that would explain the relatively lower accuracy, both in terms of comparison to multinomial and to other models.

MultinomialNB is typically used with discrete data, which is primarily the case for this dataset, explaining the slightly improved accuracy over Gaussian. However, we believe that the accuracy measure, F1-Score, is lower in comparison to other models, because naive bayes takes a probabilistic approach to prediction. In our dataset, probability may not often be meaningful when considering it just in terms of binary correlation between each feature and the education label. As an example, it is equally as likely that someone is 6 feet tall given that they have an advanced degree and that someone is 6 feet tall and holds a bachelors degree.

### Neural Network

Neural networks are often a good option for predictive models, so we decided to try that as well. We chose to one hot encode the data for neural networks to maximize the number of nodes being used at each layer as determined by the multilayer perceptron classifier's algorithm. This could allow for more intricate correlations between features to be picked up, but may also cause underfitting because of picking up unimportant correlations. We also chose to remove outliers as they could cause the loss function to evaluate such that weights would have to be shifted by a large amount.

In [35]:
scaler = preprocessing.MinMaxScaler()
nn = neural_network.MLPClassifier()
ppl = pipeline.Pipeline(steps=[('scaler', scaler), ('nn', nn)])

nn_data, nn_labels = remove_outliers(data, labels)
nn_data = one_hot_encode(nn_data)

param_grid = {'nn__activation': ['logistic', 'tanh', 'relu'], 'nn__solver': ['sgd', 'adam'], 'nn__learning_rate': ['constant', 'adaptive']}
inner = model_selection.GridSearchCV(ppl, param_grid, scoring='accuracy', cv=5, iid=True)
pred = model_selection.cross_val_predict(inner, nn_data, nn_labels, cv=5)
print('Accuracy:', metrics.accuracy_score(nn_labels, pred))
print('Weighted F1:', metrics.f1_score(nn_labels, pred, average='weighted'))

Accuracy: 0.5909962834192543
Weighted F1: 0.5264157505793133


We found an accuracy of about 59%, which was comparable to the decision tree, and our highest for the final data set. We believe that NNs worked relatively well because they weigh features that have importance with a heavier weight value, finding the essential features while diminishing the impact that a less important feature, say pets, may have in predicting education. This is different than a Decision Tree in that a Decision Tree splits upon any discrete variable, and while the end leaves may have the same label, there is a possibility that they have different predicted classes. NNs can remove that possibility through providing weights to the deemed 'essential or important' features that are more important in predictions. NNs also had the advantage of not breaking down in higher dimensions, which might have happened in some of the other models because of the large number of features in our data when exacerbated by the one hot encoding.

### Ensembles

Ensembles made sense to test last, since we now have several different classifiers which we can put together. We chose to test the random forest ensemble, the ada boost ensemble, and also the voting ensemble. The random forest and ada boost also perform voting, but they purely use decision trees as their estimators. We also decided to try the voting ensemble because we had tested the other classifiers and had seen which ones had performed better. We expect ensembles to work because of the nature of different classifiers to pick up correlations in different parts of the data set, and not be completely overlapping in which data points they predict correctly (should have a little but not a lot of overlap to be effective).

In [36]:
ensemble_data, ensemble_labels = remove_outliers(data, labels)
ensemble_data = one_hot_encode(ensemble_data)

#### Random Forest

In [37]:
param_grid = {'max_depth': [25,30,35,40], 'min_samples_leaf': [0.002,0.005,0.01], 'max_features': ('sqrt','log2')}
inner = model_selection.GridSearchCV(ensemble.RandomForestClassifier(n_estimators=100), param_grid, scoring='accuracy', cv=5)
pred = model_selection.cross_val_predict(inner, ensemble_data, ensemble_labels, cv=5)
print('Accuracy:', metrics.accuracy_score(ensemble_labels, pred))
print('Weighted F1:', metrics.f1_score(ensemble_labels, pred, average='weighted'))

Accuracy: 0.5917156216281021
Weighted F1: 0.5007874483451646


#### Ada Boost

In [38]:
param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.7, 0.85, 1.0]}
inner = model_selection.GridSearchCV(ensemble.AdaBoostClassifier(), param_grid, scoring='accuracy', cv=5)
pred = model_selection.cross_val_predict(inner, ensemble_data, ensemble_labels, cv=5)
print('Accuracy:', metrics.accuracy_score(ensemble_labels, pred))
print('Weighted F1:', metrics.f1_score(ensemble_labels, pred, average='weighted'))

Accuracy: 0.5945330296127562
Weighted F1: 0.5180353583944314


#### Voting

For the voting ensemble, we used the KNN, decision tree, neural network, and unbalanced SVC. One of the set of weights used in hyperparameters testing was estimated based on how well each classifier had performed individually during the test set phase, and another set of weights was based on final set performance.

In [39]:
estimators = [
    ('knn', pipeline.make_pipeline(preprocessing.StandardScaler(), decomposition.PCA(n_components=7), neighbors.KNeighborsClassifier())),
    ('dt', tree.DecisionTreeClassifier()),
    ('nn', pipeline.make_pipeline(preprocessing.MinMaxScaler(), neural_network.MLPClassifier())),
    ('svc', pipeline.make_pipeline(preprocessing.StandardScaler(), decomposition.PCA(n_components=7), svm.SVC(decision_function_shape='ovr')))
]

param_grid = {'weights': [[1, 1, 1, 1], [1, 0.7, 0.7, 0.5], [1, 1, 1, 0.7]]}
inner = model_selection.GridSearchCV(ensemble.VotingClassifier(estimators=estimators), param_grid, scoring='accuracy', cv=5, iid=True)
pred = model_selection.cross_val_predict(inner, ensemble_data, ensemble_labels, cv=5)
print('Accuracy:', metrics.accuracy_score(ensemble_labels, pred))
print('Weighted F1:', metrics.f1_score(ensemble_labels, pred, average='weighted'))

Accuracy: 0.566358949766215
Weighted F1: 0.5158715609560718


The accuracy of the voting ensemble turned out to be around the average of the ensembled classifiers. This is potentially indicative of the different classifiers overlapping in what classes they predict for certain points. For an ensemble, it would be preferable that the classifiers to have higher accuracy in different parts of the data so that there is a wider coverage to perform the voting over, especially for the Random Forest and the Voting classifiers. This does not seem to be the case with our dataset; there probably is a lot of uncertainty in a large proportion of the data, where no classifier is able to provide correct predictions.

## Results and Analysis

Most of our models had accuracies between 55% and 60%. Our data was primarily categorical and had to be scaled using one-hot encoding, which increased our number of features. By increasing the number of dimensions, our models and predictive algorithms fell prey to the curse of dimensionality. Additionally, we were attempting to predict one of 5 different labels, which makes it difficult to predict with a high accuracy, especially given that our data was primarily categorical, and many models perform better on binary classification problems. Given these circumstances, we believe that our predictive algorithms are relatively accurate, but not necessarily usable, and can probably be improved upon with additional data exploration and feature engineering.

In our dataset, there was a heavy class imbalance favoring 'bachelors'. This class made up about 60% of the data. Additionally, using domain knowledge of real life experiences, it seems very difficult to predict someone's education level based on factors such as religion, whether or not they smoke or drink, body type, preference for pets, and height. It is possible for someone of any religion to have any level of education and someone who is tall or short could receive their ph.d. Features such as income and job are far better indicators of education level. The problem we faced with income was that the majority of values were missing, as a lot of people probably did not want to self-report income on a dating app. The problem we faced with job/occupation was that there were a a lot of unique values, which would further worsen the curse of dimensionality. Even with the given values, the job values were very vague and didn't give any specific role details that could be used to predict education. For example, there was a value of 'entertainment'. This could include the most succesful singers in the world who make millions of dollars or a local band member who was still in high school. We also found another potential source of error could come from the fact that the data was self reported. This means that if someone was embarrased or thought a factor about themselves, such as body type, was unattractive, they would not report it. This could lead to a lot of significant or differentiating factors to be missing. Since on a dating website, people are encouraged to portray the best version of themselves to make themselves attractive to potential partners. Therefore, there
may be inaccuracies in some of the data, especially in features such as height, and body type where people
might lie, affecting the models that we use.

Another aspect that we found to complicate predicting the labels was that one of the classes was 'space camp'. We were not sure why this was an option and is it clearly did not indicate a real or serious education level, but felt that we should not remove the elements that belonged to the 'space camp' class as it was not insignificant. Given that this was a fake class, predicting this class was difficult as there are no real factors that indicate whether someone's education level was space camp. Two elements could have the exact same attributes and one could have a class of 'advanced degree' while the other could have a class of 'space camp', which simply confuses any model or algorithm.


All in all, while we were able to predict the education level around a 68% accuracy, we cannot conclude that there is necessarily a strong correlation between the features that we explored and the education level. Given the imbalance favoring 'bachelors', predicting 'bachelors' for every element would result in a similar accuracy. However, we did find that our predictions were not just 'bachelors', which was only predicted 60 - 75% of the time, and that the F1 scores were not very bad. This is a good sign that our models did fit to the data and find some level of correlation between the features and the labels and were not completely overfitted to the class imbalance.